In [1]:
import torch

import torch.nn as nn

from torch.autograd import Variable
from torchvision import datasets, transforms, utils
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
from torch.autograd import Variable

from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import time
import random

from collections import Counter
from tqdm import tqdm

In [2]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

# create a symbolic link
!ln -s /content/gdrive/My\ Drive/ /mydrive

!ls /mydrive

# navigate to /mydrive/car-damage-project
%cd /mydrive/car-damage-project

/
Mounted at /content/gdrive
 Adriana.jpg	     'FDS - Team 4.gsheet'   output.txt		   VAMR.gdoc
 car-damage-project  'IACV Cards.gdoc'	     pytorch-tutorial	   yolov4
'Colab Notebooks'    'Introduction to ML'   'USA 2019'
 drive_data	      output_resnet.txt     'VAMR Complete.gdoc'
/content/gdrive/My Drive/car-damage-project


In [7]:
# define resizing to (224, 224, 1) and load dataset
data_transforms = {
    'training': transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize([224, 224]),
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5))
    ]),
    'validation': transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize([224, 224]),
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5))
    ]),
}

data_dir = '../car-damage-project/kaggle_data/'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['training', 'validation']}

train_loader = DataLoader(dataset=image_datasets['training'], 
                          batch_size=1840, 
                          shuffle=True)

test_loader = DataLoader(dataset=image_datasets['validation'], 
                         batch_size=420, 
                         shuffle=False)

In [8]:
# Training Arguments Settings   
# Load Training Data & Testing Data
#Convert Training data to numpy
train_data = next(iter(train_loader))[0].numpy()
train_label = next(iter(train_loader))[1].numpy()

#scaler = StandardScaler()
#train_data = scaler.fit_transform(train_data)

# Print training data size
print('Training data size: ',train_data.shape)
print('Training data label size:',train_label.shape)   

# plt.imshow(train_data[0])
# plt.show()
test_data = next(iter(test_loader))[0].numpy()
test_label = next(iter(test_loader))[1].numpy()

#test_data = scaler.transform(test_data)

training_features = train_data.reshape(1840, -1)
test_features = test_data.reshape(420, -1)

# Training SVM
print('------Training and testing SVM------')
clf = svm.SVC(C=5, gamma=0.05,max_iter=100)
clf.fit(training_features, train_label)
    
    
#Test on test data
test_result = clf.predict(test_features)
precision = sum(test_result == test_label)/test_label.shape[0]
print('Test precision: ', precision)
    
#Test on Training data
train_result = clf.predict(training_features)
precision = sum(train_result == train_label)/train_label.shape[0]
print('Training precision: ', precision)
    
    
#Show the confusion matrix
matrix = confusion_matrix(test_label, test_result)

Training data size:  (1840, 1, 224, 224)
Training data label size: (1840,)
------Training and testing SVM------


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


Test precision:  0.44047619047619047
Training precision:  0.5625
